In [1]:
import pandas as pd

Merge all predictions such that shape of df is [number_of_examples_in_val_df, number_models_run+'target'] (Code not shown since too many dataframes to be downloaded from kaggle)


While many models were trained, the challenge allows kernel-only submissions which have to run from raw data to test results in 9 hours. While pretrained models can be uploaded, prredictions must be complete within 9 hours (kernel running time). Hence only a subset of models can be used in practice. Therefore, an ensemble model was fit using logistic regression with a high lasso coefficient. This allows for feature selection. Next, the df was restricted to the selected features and fit to a final logistic regression model. This model is saved and used for predicting on the test set. 

In [ ]:
#pred cols are those with predictions from models
pred_cols = [c for c in df.columns if c not in non_pred_cols]
X = df[pred_cols].values
y = df['target'].values

#fit in log-reg model with strong lasso!
clf = LogisticRegression(penalty='l1', class_weight='balanced', random_state=42, solver='liblinear', C=0.005)
clf.fit(X,y)

#select the features with the top 6 coefficients
lr_coefs = pd.Series(clf.coef_[0,:], index=pred_cols).sort_values(ascending=False)
selected_cols = list(lr_coefs.index[0:6])
print(selected_cols)

#restrict data to selected features
X_sel = df[selected_cols].values
clf_sel_final = LogisticRegression(penalty='l1', class_weight='balanced', random_state=42, solver='liblinear', C=1)
clf_sel_final.fit(X_sel,y)

In [ ]:
from sklearn.externals import joblib
joblib.dump(clf_sel_final, 'clf_sel_final.pkl')

The top 6 models selected for ensembling were:

1. **test_results_float_p2_e2** : BERT run with the entire training set for 2 epochs, with targets as float and unweighted cross entropy loss
2. **test_results_pretrained_72K_float_p1_e2_weighted-m3**: BERT pretrained on 72K of the examples and subsequently run for 1.5 epochs, with targets as float and weighted cross entropy loss with weights assigned by method3
3. **test_results_pretrained_72K_float_p2_weighted-m3**: BERT pretrained on 72K of the examples and subsequently run for 1 epoch, with targets as float and weighted cross entropy loss with weights assigned by method3
4. **test_results_float_p1_e2**: BERT run with the entire training set for 1.5 epochs, with targets as float and unweighted cross entropy loss
5. **epoch0_lstm_glove_ft_0.05_nc-m3**: LSTM with 1 epoch using glove and fasttext embeddings. Dropout was 0.05. No characters were removed from the comment_text. Loss was weighted cross entropy with weights assigned as method 3 
6. **test_results_float_p1_cased'**: Cased BERT run for 0.5 epochs with targets as float and unweighted cross entropy loss